# Correlation Heatmaps

## Trevor Rowland, Scott Campbell :: 2-4-2025

This notebook serves as a repository of functions for creating correlation heatmaps, used to identify relationships between features in our datasets. This notebook will use the `players` dataset, containing aggregated player stats for all NBA players from 2004-2024.

## 1. Importing Packages and Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_pickle('/home/arch-db/Documents/github/bint-capstone/data-sources/player-responses/new-player-data.pkl')
df.head()

,id,playerName,position,age,games,gamesStarted,minutesPg,fieldGoals,fieldAttempts,fieldPercent,...,totalRb,assists,steals,blocks,turnovers,personalFouls,points,team,season,playerId
0,10236,Marcus Camby,C,32,70,70,2369,312,659,0.473,...,816,223,87,231,122,183,785,DEN,2007,cambyma01
1,10159,Shareef Abdur-Rahim,C,30,80,45,2015,310,654,0.474,...,398,109,53,40,116,243,793,SAC,2007,abdursh01
2,10160,Hassan Adams,SG,22,61,8,495,75,135,0.556,...,77,13,17,4,22,47,174,NJN,2007,adamsha01
3,10161,Maurice Ager,SG,22,32,1,214,22,70,0.314,...,21,7,4,3,15,27,69,DAL,2007,agerma01
4,10162,LaMarcus Aldridge,C,21,63,22,1392,241,479,0.503,...,312,24,22,73,43,186,565,POR,2007,aldrila01


## 2. Removing Unnecessary Data

Now we need to remove data that will screw up the process of creating dummy variables and 